 [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239303-lesson-8-deployment)

# 部署

## 回顾 

我们已经构建了一个带记忆的智能体: 

* `act` - 让模型调用特定工具
* `observe` - 将工具输出返回给模型
* `reason` - 让模型对工具的输出进行推理以决定下一步做什么 (例如，调用另一个工具或直接响应)
* `persist state` - 用一个内存检查点来支持带有中断的长对话

## 目标

现在，我们将介绍如何将我们的智能体在本地的 Studio 和`LangGraph Cloud`上实际部署

In [3]:
%%capture --no-stderr
%pip install --quiet -U langgraph_sdk langchain_core

## 概念

有几个核心概念需要理解

`LangGraph` —
- Python 和 JavaScript 的库 
- 允许创建智能体工作流

`LangGraph API` —
- 打包图代码 
- 提供任务队列以管理异步操作
- 提供持久化功能，以维护交互间的状态

`LangGraph Cloud` --
- HLangGraph API 托管服务
- 允许从 GitHub 仓库部署图
- 还提供已部署图的监控和追踪
- 每个部署都可通过唯一的 URL 访问

`LangGraph Studio` --
- 集成开发环境（IDE）用于LangGraph应用程序
- 使用 API 作为后端，允许实时测试和探索图
- 可在本地或云部署下运行

`LangGraph SDK` --
- Python库，用于与LangGraph图进行程序化交互 
- 提供了一种在本地或云端处理图的统一接口
- 允许创建客户端、访问助手、thread管理和执行运行

## 本地测试

**⚠️ DISCLAIMER**

要在本地开发服务器上启动，请在您的终端中运行以下命令（不支持ipynb，得放到py文件里）：

```
langgraph dev
```

您应该看到以下输出：
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

打开您的浏览器并导航到 Studio UI: `https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024`.

In [1]:
if 'google.colab' in str(get_ipython()):
    raise Exception("Unfortunately LangGraph Studio is currently not supported on Google Colab")

In [2]:
from langgraph_sdk import get_client

In [ ]:
# This is the URL of the local development server
URL = "http://127.0.0.1:2024"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [4]:
assistants[-3]

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'config': {},
 'metadata': {'created_by': 'system'},
 'name': 'agent',
 'created_at': '2025-03-04T22:57:28.424565+00:00',
 'updated_at': '2025-03-04T22:57:28.424565+00:00',
 'version': 1}

In [3]:
# We create a thread for tracking the state of our run
thread = await client.threads.create()

现在，我们可以[使用`client.runs.stream`](https://langchain-ai.github.io/langgraph/concepts/low_level/#stream-and-astream) 运行我们的智能体：

* The `thread_id`
* The `graph_id`
* The `input` 
* The `stream_mode`

我们将在未来的模块中深入讨论流式传输。

目前，只需认识到我们在图的每个步骤之后使用`stream_mode="values"`[流式传输](https://langchain-ai.github.io/langgraph/cloud/how-tos/stream_values/)状态的全部值。
 
状态被捕获在 chunk.data 中。

In [4]:
from langchain_core.messages import HumanMessage

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'cdbd7bd8-c476-4ad4-8ab7-4ad9e3654267', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_iIPryzZZxRtXozwwhVtFObNO', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5'}, 'type': 'ai', 'name': None, 'id': 'run-06c7243c-426d-4c81-a113-f1335dda5fb2', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_iIPryzZZxRtXozwwhVtFObNO', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '988cb170-f6e6-43c1-82fd-309f519abe6d', 'tool_call_id': 'c

## 云端测试

我们可以通过 LangSmith 将应用部署到云上，具体操作请参考[此处说明](https://langchain-ai.github.io/langgraph/cloud/quick_start/#deploy-from-github-with-langgraph-cloud)。

### 在 GitHub 上创建一个新的仓库

* 前往你的 GitHub 账户
* 点击右上角的"+"图标，并选择`"New repository"`
* 命名你的仓库 (例如：`langchain-academy`)

### 将您的 GitHub 仓库添加为远程仓库

* 返回到您在课程开始时克隆 `langchain-academy` 的终端
* 添加您新创建的 GitHub 仓库作为远程仓库

```
git remote add origin https://github.com/your-username/your-repo-name.git
```
* 将它推送上去
```
git push -u origin main
```

### 将 LangSmith 连接到您的 GitHub 仓库

* 前往[LangSmith](hhttps://smith.langchain.com/)
* 在左侧LangSmith面板上点击`deployments`标签
* 点击 `+ New Deployment`
* 然后选择你刚才为这个课程创建的Github仓库(例如：`langchain-academy`)
* 将`LangGraph API config file`指向一个`studio`目录
* 例如：对于module-2来说使用：`module-2/studio/langgraph.json`
* 设置您的 API 密钥

![Screenshot 2024-09-03 at 11.35.12 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fd61c93d48e5d0f47_deployment2.png)

### 与您的部署一起工作

我们可以以几种不同的方式与我们的部署进行交互：

* 如前所述，使用[SDK](https://langchain-ai.github.io/langgraph/cloud/quick_start/#use-with-the-sdk)。
* 使用[LangGraph Studio](https://langchain-ai.github.io/langgraph/cloud/quick_start/#interact-with-your-deployment-via-langgraph-studio).

![Screenshot 2024-08-23 at 10.59.36 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fa159a09a51d601de_deployment3.png)

只需确保`LANGSMITH_API_KEY`被设置即可在此笔记本中使用 SDK！

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGCHAIN_API_KEY")

In [ ]:
# Replace this with the URL of your deployed graph
URL = "https://langchain-academy-8011c561878d50b1883f7ed11b32d720.default.us.langgraph.app"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [37]:
# Select the agent
agent = assistants[0]

In [38]:
agent

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'created_at': '2024-08-23T17:58:02.722920+00:00',
 'updated_at': '2024-08-23T17:58:02.722920+00:00',
 'config': {},
 'metadata': {'created_by': 'system'}}

In [40]:
from langchain_core.messages import HumanMessage

# We create a thread for tracking the state of our run
thread = await client.threads.create()

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8ea04559-f7d4-4c82-89d9-c60fb0502f21', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27'}, 'type': 'ai', 'name': None, 'id': 'run-b0ea5ddd-e9ba-4242-bb8c-80eb52466c76', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '1bf558e7-79ef-4f21-bb66-acafbd04677a', 'tool_call_id': 'c